# 0. Import libraries

In [ ]:
!pip install pandas -q
!pip install numpy -q
!pip install geopy -q
!pip install matplotlib -q
!pip install seaborn -q
!pip install scikit-learn -q

In [ ]:
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt
import sklearn
from matplotlib import colors
from scipy.stats import zscore

# 1. Load data

- upload the dataset

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
poi_df = pd.read_csv('data/poi.csv')

# 2. Add Features

In [ ]:
def NaN_analysis(df):
    # analysis1: percentage of NaN values in columns
    d = {'column': [], 'n_nan': [], 'percentage': []}
    for col in df.columns:
        n_nan = df[(df[col].isna()) | (df[col].isnull())].shape[0]
        d['column'].append(col)
        d['n_nan'].append(n_nan)
        d['percentage'].append((n_nan*100)/df.shape[0])
    analysis1 = pd.DataFrame(d).sort_values('percentage', ascending=False).reset_index(drop=True)
    # analysis2: number of NaN values in each row grouped by number of NaN values
    d = {'n_nan': [], 'n_rows': []}
    for n_nan in range(1, df.shape[1]+1):
        n_rows = df[df.isna().sum(axis=1) == n_nan].shape[0]
        d['n_nan'].append(n_nan)
        d['n_rows'].append(n_rows)
    analysis2 = pd.DataFrame(d).sort_values('n_nan', ascending=False).reset_index(drop=True)
    return analysis1, analysis2

In [ ]:
analysis1, analysis2 = NaN_analysis(poi_df)

In [ ]:
analysis1

In [ ]:
analysis2

We want to add to our dataset the feature number of point of interest. By running the NaN analysis we notice that the only columns that we can use are lat and lon, the others are mainly composed by NaNs. 

The data of the new feature will be the mean of the distances between each house and its closest point of interest. 

In [ ]:
poi_df

In [ ]:
poi_df = poi_df[['lat', 'lon']]

In [ ]:
def get_n_poi(lat, lon, poi_df, radius=0.001):
    # radius = 0.03 -> 3km
    # get all poi in a certain radius from the current lat and lon
    series = poi_df[(poi_df['lat']-lat)**2 + (poi_df['lon']-lon)**2 <= radius**2]
    # n_poi = np.nan if series.shape[0] == 0 else series.shape[0]
    return series.shape[0] # n_poi

In [ ]:
train_df['n_poi'] = train_df.apply(lambda x: get_n_poi(x['latitude'], x['longitude'], poi_df), axis=1)

In [ ]:
train_df['n_poi'].hist(bins=200)

# 3. Fill NaNs

- check which entries are incorrect for some features and then fix or drop the entries (correctness)
- solve the NaN, for each case decide if we fix or drop (completeness)
- finally we can obtein the clean dataset, with which we can start to work and analyse it in step 2

In [ ]:
train_df.info()

In [ ]:
train_df

We start by making an anysis of the NaN.

In [ ]:
analysis1, analysis2 = NaN_analysis(train_df)

In [ ]:
analysis1

In [ ]:
analysis2

In [ ]:
# drop id
train_df.drop('id', axis=1, inplace=True)

In [ ]:
def groupby_count_percentage(df, col):
    tmp = df[col].to_frame().copy()
    tmp.fillna('NaN', inplace=True)
    tmp['count'] = 0
    tmp = tmp.groupby(col).count().reset_index().sort_values('count', ascending=False).reset_index(drop=True)
    tmp['percentage'] = (tmp['count']*100)/df.shape[0]
    return tmp

We can notice that for the features balcony and garden the NaN are actually 'False'.

In [ ]:
groupby_count_percentage(train_df, 'balcony')

In [ ]:
groupby_count_percentage(train_df, 'garden')

In [ ]:
train_df['garden'] = train_df['garden'].fillna(False)
train_df['balcony'] = train_df['balcony'].fillna(False)

We encode garden, balcony and condition. In order to have for each feature only int or float as data, in this way we can plot graphs to better understand what to do with the NaN if drop or fix them. 
\
We have to point out that we encode condition with the assumption that also the test data will have the same labels as the train data for the feature condition.


In [ ]:
train_df['garden'] = train_df['garden'].apply(lambda x: 1 if x == True else 0)
train_df['balcony'] = train_df['balcony'].apply(lambda x: 1 if x == True else 0)

In [ ]:
map_conditions = {}
index = 0
for value in train_df['conditions'].unique():
    if value not in map_conditions:
        map_conditions[value] = index
        index += 1
map_conditions

In [ ]:
train_df['conditions'] = train_df['conditions'].apply(lambda x: map_conditions[x])
train_df['conditions'] = train_df['conditions'].apply(lambda x: np.nan if x == map_conditions[np.nan] else x)

For each column that has at keast a NaN as a data we plot the histogram distribution also in 2D

In [ ]:
columns_wnan = []
for col in train_df.columns:
    if train_df[train_df[col].isna()].shape[0] > 0:
        columns_wnan.append(col)
columns_wnan

In [ ]:
def get_hist(df, feature, min_value=None, max_value=None, b=100):
    df = df.dropna(subset=[feature]).reset_index(drop=True) #before plotting we remove the NaNs otherwise it will not be possible to plot the graphs 
    if min_value == None:
        min_value = df[feature].min()
    if max_value == None:
        max_value = df[feature].max()
    tmp = df[df[feature] >= min_value]
    tmp = tmp[tmp[feature] <= max_value]
    plt.hist(tmp.loc[:, feature], bins=b)
    plt.title('histogram distribution of {} (min_value: {}, max_value: {}, bins: {})'.format(feature, min_value, max_value, b))
    plt.ylabel('value')
    plt.xlabel('index')
    plt.show()
    plt.close()
    plt.hist2d(pd.Series(np.array([i for i in range(tmp.loc[:, feature].shape[0])])), tmp.loc[:, feature], bins=b, norm = colors.LogNorm())
    plt.title('2D histogram distribution of {} (min_value: {}, max_value: {}, bins: {})'.format(feature, min_value, max_value, b))
    plt.ylabel('value')
    plt.xlabel('index')
    plt.colorbar()
    plt.show()
    plt.close()
    return

In [ ]:
for col in columns_wnan:
    get_hist(train_df, col)

From the graphs we can observe that we have enough information to use the algorithm: imputation of missing values - KNN imputer for all the features which have at least a NaN. 
\
Moving our attention to the KNNImputer, it will fill the NaNs using the mean value from the 5 nearest neighbors of each NaN found in the training set for each feature considered. 

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer()
train_df = pd.DataFrame(imputer.fit_transform(train_df), columns=train_df.columns)

We run again the NaN analysis to verify that there are no more NaNs in any feature. 

In [ ]:
analysis1, analysis2 = NaN_analysis(train_df)

In [ ]:
analysis1

In [ ]:
analysis2

In [ ]:
train_df

We can notice that some NaN have been filled , by the KNN Imputer, with values which do not respect reality, for example we have that an house has 0.8 as elevator (which is neither 0 = no elevator nor 1 = yes elevator). Actually this is not a problem for the model, these data are valid for the mathematical model we are building.  

In [ ]:
import seaborn as sns
sns.heatmap(train_df.corr())

In [ ]:
train_df['ratio'] = train_df.apply(lambda x: x['n_rooms']/max(1, x['n_bathrooms']), axis=1)
train_df['m2_per_bathrooms'] = train_df.apply(lambda x: (x['surface']/max(1, x['n_rooms']*3 + x['n_bathrooms'])), axis=1)

In [ ]:
sns.heatmap(train_df.corr())

# 2. Dataset analysis

- plot the features
- remove the outliers (only on the train set) 


We plot again the histograms also in 2D as before for every feature, with the only difference that now we have assigned values to the NaNs. 

In [ ]:
for col in train_df.columns:
    get_hist(train_df, col)

In [ ]:
train_df

From the graphs we can observe that there are no outliers for the features: id, balcony, garden, conditions, latitude, longitude, n_rooms and elevator. 
\
We do a boxplot only for the features with the outliers, in order to then remove the outliers. 

In [ ]:
def boxplot(df, feature):
    plt.boxplot(df.loc[:, feature])
    plt.title('boxplot of {}'.format(feature))
    plt.ylabel('value')
    plt.show()
    plt.close()
    return

In [ ]:
for col in train_df.columns:
    boxplot(train_df, col)

In [ ]:
X_train = train_df.drop('price', axis=1)
y_train = train_df['price']

In [ ]:
# minmax scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

In [ ]:

from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
X_train = pd.DataFrame(normalizer.fit_transform(X_train), columns=train_df.columns)


In [ ]:
MMStrain_df = pd.concat([X_train, y_train], axis=1)

In [ ]:
for col in train_df.columns:
    plt.scatter(train_df[col], train_df['price'])
    plt.show()

We remove the outliers with the z score.  

In [ ]:
original_shape = train_df.shape[0]
z_scores = zscore(MMStrain_df)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 2).all(axis=1)

In [ ]:
MMStrain_df = MMStrain_df[filtered_entries]
new_shape = MMStrain_df.shape[0]

In [ ]:
print('original shape: {}'.format(original_shape))
print('new shape: {}'.format(new_shape))
print('percentage of rows removed: {}%'.format(((original_shape-new_shape)*100)/original_shape))

In [ ]:
for col in MMStrain_df.columns:
    plt.scatter(MMStrain_df[col], MMStrain_df['price'])
    plt.show()

In [ ]:
# ricalcolo lo zscore sulla singola feature e seleziono il sottoinsieme di entry che hanno zscore < 5 (tmp_df)
# quindi calcolo il minimo e il massimo di tmp_df sulla feature che sto considerando per poi 
# sostituire i valori di train_df che hanno zscore > 5, sulla feature che sto considerando, con il minimo o il massimo
'''
for col in train_df.columns:
    z_scores = zscore(train_df[col])
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 5)
    tmp_df = train_df[col][filtered_entries]
    lower, upper = tmp_df.min(), tmp_df.max()
    train_df[col] = train_df[col].apply(lambda x: lower if x < lower else upper if x > upper else x)
'''
pass

In [ ]:
for col in MMStrain_df.columns:
    get_hist(MMStrain_df, col)

In [ ]:
for col in MMStrain_df.columns:
    boxplot(MMStrain_df, col)

# 3. Feature importance analysis

- may drop features that are not relevant for our model 


In [ ]:
MMStrain_df

With the random forest regressor we will compute the features importance.

In [ ]:
# create x, y
x = MMStrain_df.drop('price', axis=1)
y = MMStrain_df['price']

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# compute feature_importances
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(x, y)
sorted(rf.feature_importances_, reverse=True)

In [ ]:
# plot feature_importances
sorted_idx = rf.feature_importances_.argsort()
plt.barh(x.columns[sorted_idx], rf.feature_importances_[sorted_idx])

We notice that the elevator is th efeature with least importance, we could drop it, but given the fact we have only 17 features we are going to keep it. 

In [ ]:
#MMStrain_df.drop(['elevator', 'balcony', 'garden', 'proximity_to_center'], axis=1, inplace=True)

# 5. Add Samples (Oversampling)

- in the case of regression we do not need to undersampling
- quindi facciamo oversampling aumentando i samples sinteticamente con funzioni apposite (SMOTE)
- oppure tramite integrazione di nuovi samples non sintetici da internet

# 5. Training

- creare tutti i regressor possibili e trainarli

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

tuned_rf = {'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'n_estimators': 500}

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'RandomForestRegressor': RandomForestRegressor(**tuned_rf),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'MLPRegressor': MLPRegressor()
}

In [ ]:
MMStrain_df.reset_index(drop=True, inplace=True)
x = MMStrain_df.drop('price', axis=1)
y = MMStrain_df['price']

In [ ]:
for model_name, model in models.items():
    print('model: {}'.format(model_name))
    model.fit(x, y)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
def evaluate_model(model, x, y):
    MAE_scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
    MSE_scores = cross_val_score(model, x, y, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
    # plot
    y_pred = cross_val_predict(model, x, y, cv=5, n_jobs=-1)
    plt.scatter(y, y_pred, alpha=0.5)
    plt.plot([0, max(y.max(), y_pred.max())], [0, max(y.max(), y_pred.max())], 'r--', lw=2)
    plt.xlabel('True price')
    plt.ylabel('Predicted price')
    plt.title('{}'.format(model))
    plt.show()
    plt.close()
    print('MAE: {:.3f} ({:.3f})'.format(-MAE_scores.mean(), MAE_scores.std()))
    print('MSE: {:.3f} ({:.3f})'.format(-MAE_scores.mean(), MAE_scores.std()))
    return MAE_scores, MSE_scores

In [ ]:
performance = {model_name: {'MSE_mean': 0.0, 'MSE_std': 0.0, 'MAE_mean': 0.0, 'MAE_std': 0.0} for model_name in models.keys()}

In [ ]:
for model_name, model in models.items():
    MAE_scores, MSE_scores = evaluate_model(model, x, y)
    performance[model_name]['MAE_mean'] = -MAE_scores.mean()
    performance[model_name]['MAE_std'] = MAE_scores.std()
    performance[model_name]['MSE_mean'] = -MSE_scores.mean()
    performance[model_name]['MSE_std'] = MSE_scores.std()

In [ ]:
performance_df = pd.DataFrame.from_dict(performance, orient='index').sort_values(by='MSE_mean').reset_index().rename(columns={'index': 'model'})
performance_df

# 6. Prediction test_df 
- performiamo i modelli trainati sul test set e quindi otteniamo le prediction per kaggle

In [ ]:
def test_df_preprocessing(test_df, map_conditions, imputer, scaler):
    test_df['n_poi'] = test_df.apply(lambda x: get_n_poi(x['latitude'], x['longitude'], poi_df), axis=1)
    test_df['garden'] = test_df['garden'].fillna(False)
    test_df['balcony'] = test_df['balcony'].fillna(False)
    test_df['garden'] = test_df['garden'].apply(lambda x: 1 if x == True else 0)
    test_df['balcony'] = test_df['balcony'].apply(lambda x: 1 if x == True else 0)
    test_df['conditions'] = test_df['conditions'].apply(lambda x: map_conditions[x])
    test_df['conditions'] = test_df['conditions'].apply(lambda x: np.nan if x == map_conditions[np.nan] else x)
    test_df['ratio'] = test_df.apply(lambda x: x['n_rooms']/max(1, x['n_bathrooms']), axis=1)
    test_df['m2_per_bathrooms'] = test_df.apply(lambda x: (x['surface']/max(1, x['n_rooms']*3 + x['n_bathrooms'])), axis=1)
    test_df = pd.DataFrame(imputer.fit_transform(test_df), columns=test_df.columns)
    #test_df.drop(['elevator', 'balcony', 'garden', 'proximity_to_center'], axis=1, inplace=True)
    test_df = pd.DataFrame(scaler.fit_transform(test_df), columns=test_df.columns)
    return test_df

In [ ]:
test_id = test_df['id']
test_df.drop('id', axis=1, inplace=True)
test_df = test_df_preprocessing(test_df, map_conditions, imputer, scaler)
test_df = pd.concat([test_id, test_df], axis=1)

In [ ]:
test_df 

In [ ]:
def submission(model, test_df, filename):
    submission_df = test_df['id'].to_frame()
    # get predictions and save them in submission_df as 'price' column (in the right format for the submission)
    submission_df['price'] = model.predict(test_df.drop('id', axis=1))
    submission_df['id'] = submission_df['id'].astype(int)
    submission_df.to_csv(filename, index=False)
    return

In [ ]:
for model_name, model in models.items():
    submission(model, test_df, 'submission_{}.csv'.format(model_name))

# 7. Tuning

- troviamo la combinazione di iperparametri migliori per il modello migliore che abbiamo fino ad ora ottenuto
- quindi accordiamo il modello migliore

In [ ]:
# tuning random forest regressor
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [200, 400, 600],
    'max_depth': [None, 10, 15],
    'max_features': [None, 'sqrt', 'log2'],
    'bootstrap': [True, False]
}
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=2)
grid_search.fit(x, y)
grid_search.best_params_

In [ ]:
tuned_model = RandomForestRegressor(**grid_search.best_params_)
tuned_model.fit(x, y)
MAE_score, MSE_score = evaluate_model(tuned_model, x, y)
print('MAE: {:.3f} ({:.3f} std)'.format(-MAE_score.mean(), MAE_score.std()))
print('MSE: {:.3f} ({:.3f} std)'.format(-MSE_score.mean(), MSE_score.std()))

submission(tuned_model, test_df, 'submission_tuned_model.csv')

In [ ]:
submission(tuned_model, test_df, 'submission_tuned_model.csv')